In [1]:
import os
import pickle
import numpy as np
import seaborn as sns
from tqdm import tqdm
import pandas as pd
# from dance_evaluation import *
import matplotlib.pyplot as plt
# from calculate_score import *
from collections import defaultdict
from utils.eval import *


In [2]:
friendly = {
    "adaptv_Bhandfoot_y":  "Both-Hand + Foot (Y)",
    "adaptv_LRfoot_xy":    "L-/R-Foot (XY)",
    "adaptv_LRfoot_res":   "L-/R-Foot (Resultant)",
    "adaptv_LRhand_xy":    "L-/R-Hand (XY)",
    "adaptv_LRhand_res":   "L-/R-Hand (Resultant)",
    "adaptv_Bfoot_x_y":    "Both-Foot (X & Y)",
    "adaptv_Bhandfoot_x":  "Both-Hand & Both-Foot (X)",
    "adaptv_Bhand_x_y":    "Both-Hand (X & Y)",
    "adaptv_Bhandfoot_res":"Both-Hand & Both-Foot (Resultant)",
}

def estimate_tempo_posvel(a, b, mode, w_sec, h_sec, tolerance=0.13):
    # Using both zero velocity and peak velocity
    segment_pairs = [
        "adaptv_Bhandfoot_y",
        "adaptv_Bhandfoot_x",
        "adaptv_LRhand_xy",
        "adaptv_LRfoot_xy",
        "adaptv_Bhand_x_y",
        "adaptv_Bfoot_x_y",
        "adaptv_LRfoot_res",
        "adaptv_LRhand_res",
        "adaptv_Bhandfoot_res",
        ]
    
    
    score_data = {}
    json_data = {}
    # oPath = f"./saved_result/tttempo_{a}_{b}/"
    root = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result_adaptive"
    pth_pos = f"{root}/tempo_{a}_{b}/pos"
    pth_vel = f"{root}/tempo_{a}_{b}/vel"
    
    bpm_dict = ["global_tempo"]
    for bpm_mode in bpm_dict:
        score_data[bpm_mode] = {}
        json_data[bpm_mode] = {}
        for seg in segment_pairs:
            fpath_pos = os.path.join(pth_pos, f"{seg}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl")
            fpath_vel = os.path.join(pth_vel, f"{seg}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl")
            
            df1 = load_pickle(fpath_pos)
            df2 = load_pickle(fpath_vel)

            # Build candidate BPM pairs 
            bpm_pos = []
            bpm_vel = []
            bpm_posvel = []
            for n in range(df1.shape[0]):
                bpm1 = df1.iloc[n][bpm_mode]   # hand (position)
                bpm2 = df2.iloc[n][bpm_mode]   # foot (position)

                bpm_posvel.append((bpm1, bpm2))
            
            ref = df1["music_tempo"].to_numpy()
            
            dts_acc3, hit_idx, ref_hit_bpm = compute_dts(ref, bpm_posvel, tau=tolerance, mode = "many")
            
            score_data[bpm_mode][friendly[seg]] = {"acc": dts_acc3, "hit_idx": hit_idx, "ref_hit_bpm": ref_hit_bpm}
            
            json_data[bpm_mode][seg] = {"bpm_pos": bpm_pos,
                                             "bpm_vel": bpm_vel,
                                            "bpm_posvel": bpm_posvel,
                                            "Acc1_bpm_pos": 5,
                                            "Acc1_bpm_vel": 5,
                                            "Acc1_bpm_posvel": dts_acc3,}
            
    
    #### Sace the score data to a pickle file
    # save_dir = f"./saved_result_adaptive/tempo_{a}_{b}/score"
    # fname1 = f"score_multi_adap_posvel_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
    # fpath1 = os.path.join(save_dir, fname1)
    # save_to_pickle(fpath1, score_data)
        
    return json_data

accuracy_dict_posvel = estimate_tempo_posvel(40, 140, "zero_uni", 5, 5/2)   # Best of two

In [ ]:
segment_pairs = [
        "adaptv_Bhandfoot_y",
        "adaptv_Bhandfoot_x",
        "adaptv_LRhand_xy",
        "adaptv_LRfoot_xy",
        "adaptv_Bhand_x_y",
        "adaptv_Bfoot_x_y",
        "adaptv_LRfoot_res",
        "adaptv_LRhand_res",
        "adaptv_Bhandfoot_res",
        ]

friendly = {
    "adaptv_Bhandfoot_y":  "Both-Hand + Foot (Y)",
    "adaptv_LRfoot_xy":    "L-/R-Foot (XY)",
    "adaptv_LRfoot_res":   "L-/R-Foot (Resultant)",
    "adaptv_LRhand_xy":    "L-/R-Hand (XY)",
    "adaptv_LRhand_res":   "L-/R-Hand (Resultant)",
    "adaptv_Bfoot_x_y":    "Both-Foot (X & Y)",
    "adaptv_Bhandfoot_x":  "Both-Hand & Both-Foot (X)",
    "adaptv_Bhand_x_y":    "Both-Hand (X & Y)",
    "adaptv_Bhandfoot_res":"Both-Hand & Both-Foot (Resultant)",
}

best_of_two={}
for seg in segment_pairs:
#     print(seg, accuracy_dict_posvel["bpm_median"][seg]["Acc1_bpm_posvel"])
    best_of_two[friendly[seg]] = accuracy_dict_posvel["global_tempo"][seg]["Acc1_bpm_posvel"]
    
best_of_two    

### For adaptive weighting

In [6]:
from collections import defaultdict

# descriptive labels for plot axis
friendly = {
    "adaptv_Bhandfoot_y":  "Both-Hand + Foot (Y)",
    "adaptv_LRfoot_xy":    "L-/R-Foot (XY)",
    "adaptv_LRfoot_res":   "L-/R-Foot (Resultant)",
    "adaptv_LRhand_xy":    "L-/R-Hand (XY)",
    "adaptv_LRhand_res":   "L-/R-Hand (Resultant)",
    "adaptv_Bfoot_x_y":    "Both-Foot (X & Y)",
    "adaptv_Bhandfoot_x":  "Both-Hand & Both-Foot (X)",
    "adaptv_Bhand_x_y":    "Both-Hand (X & Y)",
    "adaptv_Bhandfoot_res":"Both-Hand & Both-Foot (Resultant)",
}

# helper to compute accuracy and fill a dictionary
def collect_accuracies(folder, metric, tolerance=0.13):
    acc = {}
    score_data = {"global_tempo": {}}
    for fname in os.listdir(folder):
        if "zero_bi" in fname:
            continue
        
        tag = fname.split("_zero_uni")[0]
        data = load_pickle(f"{folder}/{fname}")
        ref  = data["music_tempo"].to_numpy()
        accuracy, hit_idx, ref_hit_bpm = compute_dts(ref, data["global_tempo"].to_numpy(),
                                            tau=tolerance, mode="one")
        
        acc[friendly[tag]] = round(accuracy, 2)
        score_data["global_tempo"][friendly[tag]] = {"acc": accuracy, "hit_idx": hit_idx, "ref_hit_bpm": ref_hit_bpm}
        
    # save the score data
    # save_dir = f"./saved_result_adaptive/tempo_{a}_{b}/score"
    # fname1 = f"score_multi_adap_{metric}_zero_uni_W5_H2.5_{a}_{b}.pkl"
    # fpath1 = os.path.join(save_dir, fname1)
    # save_to_pickle(fpath1, score_data)
    
    return acc, score_data


# path setup
a, b = 45, 140
root = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result_adaptive"
pth_pos = f"{root}/tempo_{a}_{b}_torso/pos"
# pth_vel = f"{root}/tempo_{a}_{b}/vel"


accuracy_dict_pos, score_data = collect_accuracies(pth_pos, "pos", tolerance=0.13)
# accuracy_dict_vel, _ = collect_accuracies(pth_vel, "vel", tolerance=0.13)

accuracy_dict_pos

{'Both-Hand + Foot (Y)': 73.38}

### Compute unique detection in half and double tempo range

In [ ]:
score_path = "/itf-fi-ml/home/sagardu/aist_tempo_COM/aist_tempo_est/saved_result/tempo_60_140/score/score_one_pos_zero_bi_W5_H2.5_60_140.pkl"

score_data = load_pickle(score_path)
hits_torso = score_data["bpm_median"]["torso_x"]["metrics_pos"]

len(hits_torso)

In [ ]:
a, b = 60, 140
_, score_data = collect_accuracies(f"{root}/tempo_{a}_{b}/pos", "pos")
hits_60_140 = score_data["bpm_median"]["Both-Hand + Foot (Y)"]["hit_idx"]
len(hits_60_140)

In [ ]:
# overlap_hips = np.intersect1d(hits_60_140, hits_torso)
# len(overlap_hips)

extra_torso = np.setdiff1d(hits_torso, hits_60_140)
len(extra_torso)

In [ ]:
a1, b1 = 25, 75
_, score_data_1 = collect_accuracies(f"{root}/tempo_{a1}_{b1}/pos", "pos")
hits_25_75 = score_data_1["bpm_median"]["Both-Hand + Foot (Y)"]["hit_idx"]

len(hits_25_75)

In [ ]:
a1, b1 = 155, 285
_, score_data_2 = collect_accuracies(f"{root}/tempo_{a1}_{b1}/pos", "pos")
hits_155_285 = score_data_2["bpm_median"]["Both-Hand + Foot (Y)"]["hit_idx"]

len(hits_155_285)

In [ ]:
import numpy as np

overlap_half = np.intersect1d(hits_60_140, hits_25_75)
overlap_double = np.intersect1d(hits_60_140, hits_155_285)
overlap_half_double = np.intersect1d(hits_25_75, hits_155_285)

print("Overlap (original ∩ half):", len(overlap_half))
print("Overlap (original ∩ double):", len(overlap_double))
print("Overlap (half ∩ double):", len(overlap_half_double))


overlap_half_ratio = len(overlap_half) / len(hits_25_75) * 100
overlap_double_ratio = len(overlap_double) / len(hits_155_285) * 100

print(f"Half-tempo overlap with original: {overlap_half_ratio:.2f}%")
print(f"Double-tempo overlap with original: {overlap_double_ratio:.2f}%")



In [ ]:
root = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result_adaptive"
a, b = 60, 140
pth_pos = f"{root}/tempo_{a}_{b}/pos"
folder = pth_pos
fname = f"adaptv_Bhandfoot_y_zero_uni_W5_H2.5_{a}_{b}.pkl"
data = load_pickle(f"{folder}/{fname}")

In [ ]:
# data["dance_genre"][not_in_60_140[0]]
# Your mapping dictionary
genre_map = {
    "gBR": "Break",
    "gPO": "Pop",
    "gLO": "Lock",
    "gMH": "Middle Hip-hop",
    "gLH": "LA style Hip-hop",
    "gHO": "House",
    "gWA": "Waack",
    "gKR": "Krump",
    "gJS": "Street Jazz",
    "gJB": "Ballet Jazz"
}

genres = data["dance_genre"].iloc[]
genre_names = genres.map(genre_map)
genre_distribution = genre_names.value_counts()

print("Genre distribution for the 25-55 BPM hits not in 60-140 BPM:")
print(genre_distribution)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# accuracy_pos = {
#     'Both-Hand & Foot (Y)': 62.04,
#     'Both-Hand & Both-Foot (X)': 28.78,
#     'L-/R-Hand (XY)': 47.28,
#     'L-/R-Foot (XY)': 45.41,
#     'Both-Hand (X & Y)': 51.16,
#     'Both-Foot (X & Y)': 51.83,
# }

# accuracy_vel = {
#     'Both-Hand & Foot (Y)': 58.02,
#     'Both-Hand & Both-Foot (X)': 26.25,
#     'L-/R-Hand (XY)': 45.19,
#     'L-/R-Foot (XY)': 35.5,
#     'Both-Hand (X & Y)': 48.77,
#     'Both-Foot (X & Y)': 43.33,
# }

# best_of_two = {
#     'Both-Hand & Foot (Y)': 71,
#     'Both-Hand & Both-Foot (X)': 42,
#     'L-/R-Hand (XY)': 61,
#     'L-/R-Foot (XY)': 56,
#     'Both-Hand (X & Y)': 63,
#     'Both-Foot (X & Y)': 62,
# }

labels   = list(accuracy_dict_pos.keys())
pos_vals = [accuracy_dict_pos[k] for k in labels]
vel_vals = [accuracy_dict_vel[k] for k in labels]
best_vals= [best_of_two[k]   for k in labels]

x         = np.arange(len(labels))
bar_width = 0.2

plt.figure(figsize=(10, 6))

bars_pos  = plt.bar(x - bar_width,      pos_vals,  bar_width, label='zero vel. onsets', color='tab:blue')
bars_vel  = plt.bar(x,                  vel_vals,  bar_width, label='peak vel. onsets', color='tab:orange')
bars_best = plt.bar(x + bar_width,      best_vals, bar_width, label='best of two',       color='tab:green')

plt.xticks(x, labels, rotation=30, ha='center')
plt.ylabel('Accuracy (%)')
plt.title('Tempo Estimation Accuracy: Combined Segments Using Adaptive Weighting')
plt.ylim(0, 100)

# Annotate
for bar in bars_pos + bars_vel + bars_best:
    h = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2,
        h + 1,
        f'{h:.1f}%',
        ha='center', va='bottom',
        fontsize=6
    )

plt.legend()
plt.tight_layout()
plt.show()


### with torso

In [25]:
friendly = {
    "adaptv_Bhandfoot_y":  "Both-Hand + Foot (Y)",

}

def estimate_tempo_posvel(a, b, mode, w_sec, h_sec):
    # Using both zero velocity and peak velocity
    segment_pairs = [
        "adaptv_Bhandfoot_y",
        ]
    
    
    score_data = {}
    json_data = {}
    # oPath = f"./saved_result/tttempo_{a}_{b}/"
    root = "/itf-fi-ml/home/sagardu/aist_tempo_est/saved_result_adaptive"
    pth_pos = f"{root}/tempo_{a}_{b}/pos"
    # pth_vel = f"{root}/tempo_{a}_{b}/vel"
    
    bpm_dict = ["bpm_avg",  "bpm_mode", "bpm_median"]
    for bpm_mode in bpm_dict:
        score_data[bpm_mode] = {}
        json_data[bpm_mode] = {}
        for seg in segment_pairs:
            fpath_pos = os.path.join(pth_pos, f"{seg}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl")
            fpath_vel = os.path.join("/itf-fi-ml/home/sagardu/aist_tempo_est/extracted_body_onsets_sept25/torso", f"torso_y_zero_uni_W5_H2.5_60_140.pkl")

            
            
            df1 = load_pickle(fpath_pos)
            df2 = load_pickle(fpath_vel)

            # Build candidate BPM pairs 
            bpm_pos = []
            bpm_vel = []
            bpm_posvel = []
            for n in range(df1.shape[0]):
                bpm1 = df1.iloc[n][bpm_mode]   # hand (position)
                bpm2 = df2.iloc[n][bpm_mode]   # foot (position)
                # bpm3 = df3.iloc[n][bpm_mode]   # hand (velocity)
                # bpm4 = df4.iloc[n][bpm_mode]   # foot (velocity)
                
                # bpm_pos.append((bpm1, bpm2))
                # bpm_vel.append((bpm3, bpm4))
                bpm_posvel.append((bpm1, 0))
            
            # music_tempo from df1 
            ref = df1["music_tempo"].to_numpy()
            
            dts_acc3, hit_idx, ref_hit_bpm = compute_dts(ref, bpm_posvel, tau=0.10, mode = "many")
            
            score_data[bpm_mode][friendly[seg]] = {"acc": dts_acc3, "hit_idx": hit_idx, "ref_hit_bpm": ref_hit_bpm}
            
            json_data[bpm_mode][seg] = {"bpm_pos": bpm_pos,
                                             "bpm_vel": bpm_vel,
                                            "bpm_posvel": bpm_posvel,
                                            "Acc1_bpm_pos": 5,
                                            "Acc1_bpm_vel": 5,
                                            "Acc1_bpm_posvel": dts_acc3,}
            
    
    #### Sace the score data to a pickle file
    # save_dir = f"./saved_result_adaptive/tempo_{a}_{b}/score"
    # fname1 = f"score_multi_adap_posvel_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
    # fpath1 = os.path.join(save_dir, fname1)
    # save_to_pickle(fpath1, score_data)
        
    return json_data

accuracy_dict_posvel = estimate_tempo_posvel(60, 140, "zero_uni", 5, 5/2)   # Best of two


In [26]:
accuracy_dict_posvel["bpm_median"]["adaptv_Bhandfoot_y"]["Acc1_bpm_posvel"]

60.99925428784489